# Analyse de base des fichiers éducation nationale. 
Objectif : voir les données, trouvé des pistes pour comprendre l'echec scolaire.

In [ ]:
# import bibliothéques d'analyse de python
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models.tools import HoverTool


In [ ]:
# import des données en RAM avec pandas
effectif_prim = pd.read_csv("donnee/fr-en-ecoles-effectifs-nb_classes.csv",sep=";")
ecole_stat = pd.read_csv("donnee/fr-en-ips-ecoles-ap2022.csv",sep=";")


## Affichage des entêtes des données
Pour les écoles primaires :

In [ ]:
effectif_prim.head(1)

In [ ]:
ecole_stat.head(1)

# premier nettoyage

Standariser les catègories public privé dans la colonne secteur.<br>
Standariser les rentrées scolaires<br>
Voir pour réduire le nombre de colonne des fichiers.<br>
colone tri dans effectif ecole<br>


In [ ]:
ecole_stat['secteur'].unique()

In [ ]:
effectif_prim['secteur'].unique()

In [ ]:
# changement dans les data ecole_stat['secteur'] et college_stat['secteur']
# Les valeurs doivent être PRIVE ou PUBLIC
ecole_stat['secteur'].replace({'public':'PUBLIC','privé sous contrat':'PRIVE'},inplace=True)

In [ ]:
effectif_prim['rentree_scolaire'].unique()

In [ ]:
ecole_stat['rentree_scolaire'].unique()

In [ ]:
# remplacer dans les données ecoles_stat colonne rentree_scolaire 2022-2023 par 2022
ecole_stat['rentree_scolaire'].replace({'2022-2023':2022},inplace=True)

In [ ]:
effectif_prim.drop(columns='tri',inplace=True)

In [ ]:
# patronyme contient des Nan remplcer par un espaces
effectif_prim['patronyme'].fillna(" ",inplace=True)

In [ ]:
effectif_prim.info()

In [ ]:
effectif_prim[effectif_prim['numero_ecole'] == '0691092M']

In [ ]:
effectif_prim[(effectif_prim.rep == 1) & (effectif_prim.rep_plus == 1)]

In [ ]:
fichier_ecole_elementaire = pd.merge(ecole_stat,effectif_prim,left_on=['rentree_scolaire','uai'],right_on=['rentree_scolaire','numero_ecole'],how='outer')

In [ ]:
fichier_ecole_elementaire.info()

In [ ]:
# supression colonne en double
fichier_ecole_elementaire.drop(columns=['academie_x','departement_x','secteur_x'],inplace=True)
# renomme les colonnes avec un _y
fichier_ecole_elementaire.rename(columns={'academie_y':'academie','departement_y':'departement','secteur_y':'secteur'},inplace=True)

## graphes France généraux école primaire :
voir l évolution des effectifs globale des effectifs en primaire<br>
voir la repartition dans les classes en 2022<br>

In [ ]:
# style graphe :
sns.set_theme(style="whitegrid")
#plt.style.use("dark_background")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
palette = sns.color_palette().as_hex()
# bokeh
output_notebook()

In [ ]:
france_ecole = fichier_ecole_elementaire.groupby(by='rentree_scolaire').sum(numeric_only=True)

france_ecole_mean = fichier_ecole_elementaire.groupby(by='rentree_scolaire').agg({"effectifs":"mean",'ips':'mean'},numeric_only=True)
france_ecole.reset_index(inplace=True)
france_ecole_mean.reset_index(inplace=True)
france_ecole = pd.merge(france_ecole,france_ecole_mean,on=['rentree_scolaire'],how='outer')
france_ecole.drop(columns=['effectifs_x','ips_x'],inplace=True)
france_ecole.rename(columns={'effectifs_y':'effectifs','ips_y':'ips'},inplace=True)
del france_ecole_mean

In [ ]:
france_ecole["pourcentage_ulis"] = france_ecole["nombre_eleves_ulis"] / france_ecole["nombre_total_eleves"] *100
france_ecole["ulis_moyen_classe"] = france_ecole["nombre_eleves_ulis"] / france_ecole["nombre_total_classes"]

In [ ]:
france_ecole.head()

In [ ]:
france_ecole["Nombre_eleves_par_classe_moyenne"] = france_ecole['nombre_total_eleves'] / france_ecole['nombre_total_classes']

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='nombre_total_eleves',ax=ax)
ax.set_ylim(0)
ax.ticklabel_format(axis='y',style='plain')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ",")))
ax.set_title("nombre d'éleve total par an")
ax.set_xlabel("année")
ax.set_ylabel("nombre total d'éleves")
plt.show()


Le nombre d'éleve baisse depuis 4 ans.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='nombre_total_classes',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre de classe de 2019 à 2022")
ax.set_xlabel("année")
ax.set_ylabel("nombre de classe")
ax.set_ylim(240000)
plt.show()

Le nombre de classe augmente. 

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(8,8)
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='Nombre_eleves_par_classe_moyenne',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre moyen d'éléve par classe entre 2019 et 2022")
ax.set_xlabel("annéé")
ax.set_ylabel("nombre moyen déleve par classe")
ax.set_ylim()
plt.show()

Le nombre d´éléve diminue et le nombre de classe augmente. Sans surprise le nombre moyen d'éléve par classe diminue.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='rep',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d'établissements REP en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("Nombre d'établissements REP")
ax.set_ylim(0)
plt.show()

Le nombre d'établissement en wone REP baisse légérement.

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(8,8)
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='rep_plus',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d'établissement REP+ en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("nombre établissement REP+")
ax.set_ylim(0,3000)
plt.show()

Le nombre d'établissement en zone REP + est stable.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='nombre_eleves_ulis',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d'éléves Ulis en France entre 2019 et 2022")
ax.set_xlabel("année")
ax.set_ylabel("Nombre d'éléves Ulis")
ax.set_ylim(0)
plt.show()

Le nombre d'éléves Ulis augmente depuis 4 ans.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='ulis_moyen_classe',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre moyen d'éléves Ulis en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("Nombre moyen d'éléves Ulis")
ax.set_ylim(0)
plt.show()

Environ une classe sur 6 à un éléve Ulis.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole,x='rentree_scolaire',y='pourcentage_ulis',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("pourcentage Ulis en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("Pourcentage moyen d'éléves Ulis")
ax.set_ylim(0)
plt.show()

Les ulis representent environ 1 éléve sur 100. 

In [ ]:
#ce limité a 2022 et voir les ulis dans les classes
# donnée non dispo ?

In [ ]:

fig,ax = plt.subplots()
sns.barplot(data=france_ecole, y="ips",ax=ax,estimator="sum",errorbar=None)
ax.set_title("Indice de Position Sociale moyen en 2022",pad=15)
ax.set_ylabel("Valeur IPS")
ax.bar_label(ax.containers[0], fontsize=10)
ax.set_ylim(0)
plt.show()

Conclusion France général: <br>
Le nombre d'éléve par classe à baisser en France. Cela est du en partie à une baisse du nombre d'éléve, mais aussi à l'augmentation du nombre de classe.
Il faut aussi noter une augmentation du nombre d'éléve Ulis qui nécéssite plus d'attention. 

In [ ]:
# libéré un peu de mémoire. 
del france_ecole

# Analyse générale public/privée en France

In [ ]:
# creer un fichier france privée_public
france_ecole_public_prive_mean = fichier_ecole_elementaire.groupby(by=['rentree_scolaire','secteur']).agg({"effectifs":"mean",'ips':'mean'},numeric_only=True)
france_ecole_public_prive = fichier_ecole_elementaire.groupby(by=['rentree_scolaire','secteur']).sum(numeric_only=True)
france_ecole_public_prive.reset_index(inplace=True)
france_ecole_public_prive_mean.reset_index(inplace=True)
france_ecole_public_prive = pd.merge(france_ecole_public_prive,france_ecole_public_prive_mean,on=['rentree_scolaire','secteur'],how='outer')
france_ecole_public_prive.drop(columns=['effectifs_x','ips_x'],inplace=True)
france_ecole_public_prive.rename(columns={'effectifs_y':'effectifs','ips_y':'ips'},inplace=True)
del france_ecole_public_prive_mean

In [ ]:
france_ecole_public_prive["pourcentage_ulis"] = france_ecole_public_prive["nombre_eleves_ulis"] / france_ecole_public_prive["nombre_total_eleves"] *100
france_ecole_public_prive["ulis_moyen_classe"] = france_ecole_public_prive["nombre_eleves_ulis"] / france_ecole_public_prive["nombre_total_classes"]
france_ecole_public_prive["Nombre_eleves_par_classe_moyenne"] = france_ecole_public_prive['nombre_total_eleves'] / france_ecole_public_prive['nombre_total_classes']

In [ ]:
france_ecole_public_prive.head(10)

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='nombre_total_eleves',hue="secteur",ax=ax)
ax.set_ylim(0)
ax.ticklabel_format(axis='y',style='plain')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ",")))
ax.set_title("nombre d'éleve total par an")
ax.set_xlabel("année")
ax.set_ylabel("nombre total d'éleves")
plt.show()

Le secteur public connait une plus forte baisse du nombre d'éléve que le privée.<br>
Le secteur privée reste relativement stable mais en légére baisse<br>

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='nombre_total_classes',hue="secteur",ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre de classe de 2019 à 2022")
ax.set_xlabel("année")
ax.set_ylabel("nombre de classe")
ax.set_ylim(0)
plt.show()

Le nombre de classe en seceur privée est relativement stable. Le secteur public connais une légére augmentation du nombre de classe.

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(8,8)
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='Nombre_eleves_par_classe_moyenne',hue="secteur",ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre moyen d'éléve par classe entre 2019 et 2022")
ax.set_xlabel("annéé")
ax.set_ylabel("nombre moyen déleve par classe")
ax.set_ylim()
plt.show()

Les éléves sont plus nombreux dans les classes du privée que dans les classes du public.<br>
On peux noter une chute du nombre d'éléve dans les 2 secteur. Mais une chute plus forte dans le public.<br> 

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(8,8)
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='rep',hue='secteur',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d'établissement REP en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("nombre établissement REP")
ax.set_ylim()
plt.show()

Pas de REP en privée.

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(8,8)
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='rep_plus',hue='secteur',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d'établissement REP+ en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("nombre établissement REP+")
ax.set_ylim()
plt.show()

pas de REP+ en privée.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='nombre_eleves_ulis',hue="secteur",ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d'éléves Ulis en France entre 2019 et 2022")
ax.set_xlabel("année")
ax.set_ylabel("Nombre d'éléves Ulis")
ax.set_ylim(0)
plt.show()

Il y a des éléves Ulis dans le privée et dans le public

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='ulis_moyen_classe',hue='secteur',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("nombre d´éléve Ulis moyen par Classe entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("nombre moyen d'éléves Ulis")
ax.set_ylim(0)
plt.show()

Mais dans une proportion moindre. Il y a 2 fois plus d'éléve Ulis dans le public que dans le privée.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(data=france_ecole_public_prive,x='rentree_scolaire',y='pourcentage_ulis',hue='secteur',ax=ax)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_title("pourcentage Ulis en France entre 2019 et 2022",pad=15)
ax.set_xlabel("année")
ax.set_ylabel("Pourcentage moyen d'éléves Ulis")
ax.set_ylim(0)
plt.show()

Nous avons donc 2 fois moins de chance d'avoir un éléves Ulis dans le privée que dans le public.

In [ ]:
fig,ax = plt.subplots()
sns.barplot(data=france_ecole_public_prive,x='secteur', y="ips",ax=ax,estimator="sum",errorbar=None)
ax.set_title("Indice de Position Sociale moyen en 2022",pad=15)
ax.set_ylabel("Valeur IPS")
ax.bar_label(ax.containers[0], fontsize=10)
ax.set_ylim(0)
plt.show()

Sans grande surprise, dans le privée le niveau social est plus haut.

In [ ]:
fichier_ecole_elementaire.info()

In [ ]:
sns.scatterplot(data=fichier_ecole_elementaire,x='ips',y='nombre_eleves_ulis',hue='secteur' )

In [ ]:
fichier_ecole_elementaire['Nombre_eleves_par_classe_moyenne'] = fichier_ecole_elementaire['nombre_total_eleves'] / fichier_ecole_elementaire['nombre_total_classes']
fichier_ecole_elementaire["ulis_moyen_classe"] = fichier_ecole_elementaire["nombre_eleves_ulis"] / fichier_ecole_elementaire["nombre_total_classes"]

In [ ]:
sns.scatterplot(data=fichier_ecole_elementaire,x='ips',y='Nombre_eleves_par_classe_moyenne',hue='secteur' )

In [ ]:
fichier_ecole_elementaire.to_csv('donnee_temp/ecole.csv',index=False)